# Logistic Regression Model

In [16]:
import numpy as np

def calculate_iqs_phased(true_haplotypes, imputed_haplotypes, threshold=0.5):
    """
    Calculate the Imputation Quality Score (IQS) for phased haplotypes.

    Args:
        true_haplotypes (numpy.ndarray): 2D array of true haplotypes, where each haplotype is represented by binary values (0 or 1).
        imputed_haplotypes (numpy.ndarray): 2D array of imputed haplotypes, where each haplotype is represented by floating-point values between 0 and 1.
        threshold (float): Threshold for converting imputed probabilities to binary values (default: 0.5).

    Returns:
        float: Imputation Quality Score (IQS).
    """
    # Check if the shapes of true and imputed haplotypes are the same
    if true_haplotypes.shape != imputed_haplotypes.shape:
        raise ValueError("Shape of true haplotypes and imputed haplotypes must be the same.")

    # Convert imputed probabilities to binary values based on the threshold
    imputed_binary = (imputed_haplotypes >= threshold).astype(int)

    # Create a contingency table
    contingency_table = np.zeros((2, 2), dtype=int)

    # Fill the contingency table
    for true_hap, imputed_hap in zip(true_haplotypes, imputed_binary):
        for true_allele, imputed_allele in zip(true_hap, imputed_hap):
            contingency_table[int(true_allele), int(imputed_allele)] += 1

    # Calculate the total number of alleles
    total_alleles = np.sum(contingency_table)

    # Calculate the observed agreement (Po)
    po = np.sum(np.diag(contingency_table)) / total_alleles

    # Calculate the expected agreement by chance (Pc)
    true_counts = np.sum(contingency_table, axis=1)
    imputed_counts = np.sum(contingency_table, axis=0)
    pc = np.sum(true_counts * imputed_counts) / (total_alleles ** 2)

    # Calculate the Imputation Quality Score (IQS)
    iqs = (po - pc) / (1 - pc)

    return iqs

In [40]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.metrics import r2_score as sklearn_r2_score
import optuna
from matplotlib import pyplot as plt
import os
import csv

# Load and preprocess the data
data_directory = '../../Data/Filtered_phased_training_data/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
accuracies = []
precisions = []
recalls = []
false_positive_rates = []
auc_rocs = []
r2_scores = []
iqs_scores = []

# Create folders for saving files
output_folder = "../../Data/model_results/logistic_regression/"
model_folder = output_folder + "models/"
csv_folder = output_folder + "csv_files/"
curve_folder = output_folder + "roc_curves/"

os.makedirs(model_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(curve_folder, exist_ok=True)

for chromosome_number in range(start, 23):
    # Create subfolders for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"
    chr_curve_folder = curve_folder + f"chr{chromosome_number}/"

    os.makedirs(chr_model_folder, exist_ok=True)
    os.makedirs(chr_csv_folder, exist_ok=True)
    os.makedirs(chr_curve_folder, exist_ok=True)

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_split.parquet"
    data = pd.read_parquet(file_name)


    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*Unknown)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='Unknown').values, dtype=torch.float32)

    print("Unknown PRS313 SNPs: ", y.shape[1])
    print("Known PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_Known" in col]].shape[1])
    print("23AndMe SNPs with LD to Unknown PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_" not in col]].shape[1])
    print("Total SNPs used for Training: ", X.shape[1])

    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the logistic regression model with lasso regularization
    class LogisticRegression(nn.Module):
        def __init__(self, input_dim, output_dim, l1_coef=0.0):
            super(LogisticRegression, self).__init__()
            self.linear = nn.Linear(input_dim, output_dim)
            self.sigmoid = nn.Sigmoid()
            self.l1_coef = l1_coef

        def forward(self, x):
            out = self.linear(x)
            out = self.sigmoid(out)
            return out

        def l1_loss(self):
            return self.l1_coef * torch.norm(self.linear.weight, p=1)
        
    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the hyperparameters for tuning
    input_dim = X_train_val.shape[1]
    output_dim = y_train_val.shape[1]
    num_epochs = 500
    batch_size = 128

    # Define the objective function for Optuna with cross-validation and early stopping
    def objective(trial):
        learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
        l1_coef = trial.suggest_float('l1_coef', 1e-5, 1e-1, log=True)
        patience = trial.suggest_int('patience', 5, 20)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])

        model = LogisticRegression(input_dim, output_dim, l1_coef).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.BCELoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=False)

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        fold_losses = []

        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_val, y_train_val.argmax(dim=1))):
            X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
            y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

            train_dataset = TensorDataset(X_train, y_train)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

            best_val_loss = float('inf')
            counter = 0

            for epoch in range(num_epochs):
                train_loss = 0.0
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y) + model.l1_loss()

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()

                train_loss /= len(train_loader)

                val_dataset = TensorDataset(X_val, y_val)
                val_loader = DataLoader(val_dataset, batch_size=batch_size)

                with torch.no_grad():
                    val_loss = 0.0
                    for batch_X, batch_y in val_loader:
                        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                        outputs = model(batch_X)
                        loss = criterion(outputs, batch_y) + model.l1_loss()
                        val_loss += loss.item()

                    val_loss /= len(val_loader)
                    scheduler.step(val_loss)

                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        counter = 0
                    else:
                        counter += 1

                    if counter >= patience:
                        # print(f"Early stopping at epoch {epoch+1}")
                        break

            fold_losses.append(best_val_loss)

        return np.mean(fold_losses)

    # Create the "optuna_studies" folder if it doesn't exist
    os.makedirs("optuna_studies", exist_ok=True)

    # Create an Optuna study and optimize the hyperparameters
    study_name = f"chr{chromosome_number}_study"
    storage_name = f"sqlite:///optuna_studies/{study_name}.db"

    # Check if the study exists

    current_dir = os.getcwd()
    study_exists = os.path.exists(current_dir + f"/optuna_studies/{study_name}.db")
    
    if study_exists:
        # Load the existing study
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    else:
        # Create a new study
        study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name)

    study.optimize(objective, n_trials=50, n_jobs=-1)

    # Print the best hyperparameters and best value
    print(f"Chr {chromosome_number} - Best hyperparameters: {study.best_params}")
    print(f"Chr {chromosome_number} - Best value: {study.best_value:.4f}")

    # Train the final model with the best hyperparameters and early stopping
    best_learning_rate = study.best_params['learning_rate']
    best_l1_coef = study.best_params['l1_coef']
    best_patience = study.best_params['patience']
    best_batch_size = study.best_params['batch_size']

    model = LogisticRegression(input_dim, output_dim, best_l1_coef).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_learning_rate)
    criterion = nn.BCELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=False)

    train_dataset = TensorDataset(X_train_val, y_train_val)
    train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)

    best_train_loss = float('inf')
    counter = 0

    for epoch in range(num_epochs):
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            outputs = model(batch_X)
            loss = criterion(outputs, batch_y) + model.l1_loss()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}")

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            counter = 0
        else:
            counter += 1

        if counter >= best_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        scheduler.step(train_loss)

    # Save the final model
    model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Final model saved at: {model_save_path}")

    # Evaluate the final model on the test set
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_preds = (test_outputs > 0.5).float()
        test_accuracy = float(((test_preds > 0.5) == y_test).float().mean())
        test_precision = precision_score(y_test.cpu().numpy(), test_preds.cpu().numpy(), average='micro')
        test_recall = recall_score(y_test.cpu().numpy(), test_preds.cpu().numpy(), average='micro')
        test_f1 = f1_score(y_test.cpu().numpy(), test_preds.cpu().numpy(), average='micro')
        test_roc_auc = roc_auc_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), average='micro')
        test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_iqs = calculate_iqs_phased(y_test.cpu().numpy(), test_outputs.cpu().numpy())

        # Calculate false positive rate
        cm = confusion_matrix(y_test.cpu().numpy().ravel(), test_preds.cpu().numpy().ravel())
        tn, fp, fn, tp = cm.ravel()
        test_fpr = fp / (fp + tn)

        # Append performance metrics to the lists
        accuracies.append(test_accuracy)
        precisions.append(test_precision)
        recalls.append(test_recall)
        false_positive_rates.append(test_fpr)
        auc_rocs.append(test_roc_auc)
        r2_scores.append(test_r2)
        iqs_scores.append(test_iqs)

        # Calculate individual R^2 scores for each SNP
        individual_r2_scores = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), multioutput='raw_values')

        # Calculate individual IQS scores for each SNP
        individual_iqs_scores = np.array([calculate_iqs_phased(y_test.cpu().numpy()[:, i].reshape(-1, 1), test_outputs.cpu().numpy()[:, i].reshape(-1, 1)) for i in range(y_test.shape[1])])

        # Get the names of the SNPs from the original dataframe
        snp_names = data.filter(regex='Unknown').columns

        # Save individual R^2 scores to a CSV file
        csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'R2 Score'])
            for snp, r2_score in zip(snp_names, individual_r2_scores):
                writer.writerow([snp, r2_score])

        print(f"Individual R^2 scores saved at: {csv_file}")

        # Save individual IQS scores to a CSV file
        iqs_csv_file = chr_csv_folder + f'individual_iqs_scores_chr{chromosome_number}.csv'

        with open(iqs_csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'IQS Score'])
            for snp, iqs_score in zip(snp_names, individual_iqs_scores):
                writer.writerow([snp, iqs_score])

        print(f"Individual IQS scores saved at: {iqs_csv_file}")

        # Save individual AUC ROC curves for each SNP
        for i, snp in enumerate(snp_names):
            try: 
                fpr, tpr, _ = roc_curve(y_test.cpu().numpy()[:, i], test_outputs.cpu().numpy()[:, i])
                plt.figure()
                plt.plot(fpr, tpr, label=f'AUC ROC = {roc_auc_score(y_test.cpu().numpy()[:, i], test_outputs.cpu().numpy()[:, i]):.4f}')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title(f'AUC ROC Curve - {snp}')
                plt.legend()
                
                curve_file = chr_curve_folder + f'auc_roc_curve_{snp}_chr{chromosome_number}.png'
                plt.savefig(curve_file)
                plt.close()
            except ValueError:
                # Save a placeholder image if there is insufficient data
                plt.figure()
                plt.axis('off')
                plt.text(0.5, 0.5, "Insufficient data for ROC curve", ha='center', va='center')
                curve_file = chr_curve_folder + f'auc_roc_curve_{snp}_chr{chrom0osome_number}.png'
                plt.savefig(curve_file)
                plt.close()

                print(f"Skipping SNP {snp} due to insufficient data")


        print(f"Individual AUC ROC curves saved in: {curve_folder}")

        # Create a DataFrame to store the performance metrics for each chromosome
        performance_df = pd.DataFrame({
            'Chromosome': list(range(start, chromosome_number + 1)),
            'Accuracy': accuracies,
            'Precision': precisions,
            'Recall': recalls,
            'False Positive Rate': false_positive_rates,
            'AUC ROC': auc_rocs,
            'R2 Score': r2_scores,
            'IQS Score': iqs_scores
        })

        # Save the performance metrics to a CSV file
        performance_csv_file = csv_folder + 'performance_metrics.csv'
        performance_df.to_csv(performance_csv_file, index=False)
        print(f"Performance metrics saved at: {performance_csv_file}")

Unknown PRS313 SNPs:  40
Known PRS313 SNPs:  20
23AndMe SNPs with LD to Unknown PRS313 SNPs:  1700
Total SNPs used for Training:  1720


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packa

Chr 1 - Best hyperparameters: {'learning_rate': 0.01823591759267881, 'l1_coef': 1.0049346589997416e-05, 'patience': 9, 'batch_size': 32}
Chr 1 - Best value: 0.0865
Epoch [1/500], Train Loss: 0.6078
Epoch [2/500], Train Loss: 0.2365
Epoch [3/500], Train Loss: 0.1788
Epoch [4/500], Train Loss: 0.1473
Epoch [5/500], Train Loss: 0.1346
Epoch [6/500], Train Loss: 0.1262
Epoch [7/500], Train Loss: 0.1224
Epoch [8/500], Train Loss: 0.1164
Epoch [9/500], Train Loss: 0.1078
Epoch [10/500], Train Loss: 0.1023
Epoch [11/500], Train Loss: 0.1006
Epoch [12/500], Train Loss: 0.1012
Epoch [13/500], Train Loss: 0.0985
Epoch [14/500], Train Loss: 0.1017
Epoch [15/500], Train Loss: 0.1041
Epoch [16/500], Train Loss: 0.0979
Epoch [17/500], Train Loss: 0.0983
Epoch [18/500], Train Loss: 0.0929
Epoch [19/500], Train Loss: 0.0936
Epoch [20/500], Train Loss: 0.0987
Epoch [21/500], Train Loss: 0.0966
Epoch [22/500], Train Loss: 0.0973
Epoch [23/500], Train Loss: 0.1004
Epoch [24/500], Train Loss: 0.0996
Epoch

[I 2024-04-29 15:42:01,660] Trial 1 finished with value: 0.42094608396291733 and parameters: {'learning_rate': 0.01961288896675574, 'l1_coef': 0.001601740272418487, 'patience': 9, 'batch_size': 128}. Best is trial 1 with value: 0.42094608396291733.
[I 2024-04-29 15:42:15,036] Trial 6 finished with value: 0.48818083852529526 and parameters: {'learning_rate': 0.009324259434815622, 'l1_coef': 0.0028534946619540092, 'patience': 11, 'batch_size': 128}. Best is trial 1 with value: 0.42094608396291733.
[I 2024-04-29 15:42:18,948] Trial 3 finished with value: 0.5323898077011109 and parameters: {'learning_rate': 0.07858175246218102, 'l1_coef': 0.015365060583572477, 'patience': 12, 'batch_size': 128}. Best is trial 1 with value: 0.42094608396291733.
[I 2024-04-29 15:42:23,012] Trial 5 finished with value: 0.532659575343132 and parameters: {'learning_rate': 0.018765258160924867, 'l1_coef': 0.045659657991690895, 'patience': 8, 'batch_size': 128}. Best is trial 1 with value: 0.42094608396291733.
[I

Chr 2 - Best hyperparameters: {'learning_rate': 0.008236314117298912, 'l1_coef': 1.0089572204728918e-05, 'patience': 10, 'batch_size': 32}
Chr 2 - Best value: 0.1196
Epoch [1/500], Train Loss: 0.4539
Epoch [2/500], Train Loss: 0.2691
Epoch [3/500], Train Loss: 0.2185
Epoch [4/500], Train Loss: 0.1887
Epoch [5/500], Train Loss: 0.1690
Epoch [6/500], Train Loss: 0.1574
Epoch [7/500], Train Loss: 0.1471
Epoch [8/500], Train Loss: 0.1398
Epoch [9/500], Train Loss: 0.1343
Epoch [10/500], Train Loss: 0.1292
Epoch [11/500], Train Loss: 0.1226
Epoch [12/500], Train Loss: 0.1189
Epoch [13/500], Train Loss: 0.1166
Epoch [14/500], Train Loss: 0.1141
Epoch [15/500], Train Loss: 0.1116
Epoch [16/500], Train Loss: 0.1105
Epoch [17/500], Train Loss: 0.1064
Epoch [18/500], Train Loss: 0.1053
Epoch [19/500], Train Loss: 0.1036
Epoch [20/500], Train Loss: 0.1017
Epoch [21/500], Train Loss: 0.1013
Epoch [22/500], Train Loss: 0.0990
Epoch [23/500], Train Loss: 0.0982
Epoch [24/500], Train Loss: 0.0967
Epo

...

[I 2024-04-29 20:49:53,613] A new study created in RDB with name: chr22_study


Individual AUC ROC curves saved in: ../../Data/model_results/logistic_regression/roc_curves/
Performance metrics saved at: ../../Data/model_results/logistic_regression/csv_files/performance_metrics.csv
Unknown PRS313 SNPs:  16
Known PRS313 SNPs:  6
23AndMe SNPs with LD to Unknown PRS313 SNPs:  772
Total SNPs used for Training:  778


[I 2024-04-29 20:51:35,386] Trial 6 finished with value: 0.3216399495418255 and parameters: {'learning_rate': 0.0036843250580409466, 'l1_coef': 0.004277796589249102, 'patience': 10, 'batch_size': 32}. Best is trial 6 with value: 0.3216399495418255.
[I 2024-04-29 20:51:50,377] Trial 2 finished with value: 0.36244591176509855 and parameters: {'learning_rate': 0.021896072433815455, 'l1_coef': 0.09018192881124869, 'patience': 18, 'batch_size': 256}. Best is trial 6 with value: 0.3216399495418255.
[I 2024-04-29 20:52:49,264] Trial 3 finished with value: 0.23542637079954148 and parameters: {'learning_rate': 0.002061411685453651, 'l1_coef': 0.001157162313014243, 'patience': 7, 'batch_size': 256}. Best is trial 3 with value: 0.23542637079954148.
[I 2024-04-29 20:52:54,122] Trial 1 finished with value: 0.08027062576550704 and parameters: {'learning_rate': 0.0597263960351438, 'l1_coef': 2.5664944761999238e-05, 'patience': 13, 'batch_size': 32}. Best is trial 1 with value: 0.08027062576550704.
[I

Chr 22 - Best hyperparameters: {'learning_rate': 0.02943043957881216, 'l1_coef': 1.0232605917875621e-05, 'patience': 12, 'batch_size': 32}
Chr 22 - Best value: 0.0679
Epoch [1/500], Train Loss: 0.3801
Epoch [2/500], Train Loss: 0.1965
Epoch [3/500], Train Loss: 0.1698
Epoch [4/500], Train Loss: 0.1514
Epoch [5/500], Train Loss: 0.1297
Epoch [6/500], Train Loss: 0.0986
Epoch [7/500], Train Loss: 0.0921
Epoch [8/500], Train Loss: 0.0858
Epoch [9/500], Train Loss: 0.0784
Epoch [10/500], Train Loss: 0.0793
Epoch [11/500], Train Loss: 0.0795
Epoch [12/500], Train Loss: 0.0724
Epoch [13/500], Train Loss: 0.0716
Epoch [14/500], Train Loss: 0.0735
Epoch [15/500], Train Loss: 0.0705
Epoch [16/500], Train Loss: 0.0696
Epoch [17/500], Train Loss: 0.0672
Epoch [18/500], Train Loss: 0.0664
Epoch [19/500], Train Loss: 0.0628
Epoch [20/500], Train Loss: 0.0620
Epoch [21/500], Train Loss: 0.0613
Epoch [22/500], Train Loss: 0.0617
Epoch [23/500], Train Loss: 0.0631
Epoch [24/500], Train Loss: 0.0694
Ep

<Figure size 640x480 with 0 Axes>

In [ ]:
import numpy as np

def calculate_iqs_unphased(true_genotypes, imputed_genotypes, threshold=0.5):
    """
    Calculate the Imputation Quality Score (IQS) for unphased genotypes.

    Args:
        true_genotypes (numpy.ndarray): 2D array of true genotypes, where each genotype is represented by values 0, 1, or 2.
        imputed_genotypes (numpy.ndarray): 2D array of imputed genotypes, where each genotype is represented by floating-point values between 0 and 2.
        threshold (float): Threshold for converting imputed probabilities to discrete genotypes (default: 0.5).

    Returns:
        float: Imputation Quality Score (IQS).
    """
    # Check if the shapes of true and imputed genotypes are the same
    if true_genotypes.shape != imputed_genotypes.shape:
        raise ValueError("Shape of true genotypes and imputed genotypes must be the same.")

    # Convert imputed probabilities to discrete genotypes based on the threshold
    imputed_discrete = np.round(imputed_genotypes).astype(int)

    # Create a contingency table
    contingency_table = np.zeros((3, 3), dtype=int)

    # Fill the contingency table
    for true_geno, imputed_geno in zip(true_genotypes, imputed_discrete):
        for true_allele, imputed_allele in zip(true_geno, imputed_geno):
            contingency_table[int(true_allele), int(imputed_allele)] += 1

    # Calculate the total number of alleles
    total_alleles = np.sum(contingency_table)

    # Calculate the observed agreement (Po)
    po = np.sum(np.diag(contingency_table)) / total_alleles

    # Calculate the expected agreement by chance (Pc)
    true_counts = np.sum(contingency_table, axis=1)
    imputed_counts = np.sum(contingency_table, axis=0)
    pc = np.sum(true_counts * imputed_counts) / (total_alleles ** 2)

    # Calculate the Imputation Quality Score (IQS)
    iqs = (po - pc) / (1 - pc)

    return iqs